In [870]:
from time import sleep

import numpy as np
from numba import njit, objmode, cfunc
from numba.types import string, List, Function
from numba import int16, int8, boolean
from numba.experimental import jitclass
from numba.typed import Dict
from numba import types

spec = [
    ('running', boolean),
    ('registers', int8[:]),
    ('index', int16),
    ('counter', int16),
    ('stack', List(int16)),
    ('memory', int8[:]),
    ('rom', string),
    ('flag', int8),
    ('cyclefreq', int8),
    ('cyclesleep', int8),
    ('instructions', types.DictType(*(int16,string)))
]

@njit
def chop(val, numbits):
    return val & int(2**numbits - 1)

@njit
def catbytes(upper, lower):
    uppermask = 0b11110000
    lowermask = 0b00001111
    return (upper & uppermask) | (lower & lowermask)

@jitclass(spec)
class ChipPure:
    def __init__(self):
        self.running: Bool = True
        self.registers: np.array[np.int8] = np.zeros(16, np.int8)
        self.index: np.int16 = np.int16(0)
        self.counter: np.int16 = np.int16(0x200)
        self.stack: List[np.int16] = [np.int16(0)]
        self.memory: np.array[np.int8] = np.zeros(1024*4, np.int8)
        self.rom: str = ""
        self.flag = self.registers[0xF]
        self.cyclefreq = 500
        self.cyclesleep = 10
        self.instructions = Dict.empty(*(int16,string))
        self.instructions[0x00E0] = 'cls'
        self.instructions[0x00EE] = 'ret'
        self.instructions[0xF007] = 'ldreg'
        self.instructions[0xC000] = 'rnd'
        
    def rnd(self, reg, mask):
        self.registers[reg] = np.random.randint(256) & mask
        
    def jp(self, target):
        self.counter = target
        
    def jpv0(self, target):
        self.counter = target + self.registers[0x0]
    
    def call(self, target):
        self.stack.append(counter)
        self.counter = target
        
    def ret(self):
        self.counter = self.stack.pop()
    
    def setf(self, val):
        self.flag = val
        
    def reset(self):
        self.memory.fill(0)
        self.registers.fill(0)
        with objmode():
            f = open(self.rom, "rb")
            f.seek(0x200)
            for n in range(len(self.memory)):
                read = f.read(1)
                if read:
                    self.memory[n] = np.frombuffer(read, dtype=np.int8)
            f.close()
        self.running = True
        self.counter = np.int16(0x200)
        self.stack.fill(0)
        
    def loadrom(self, rom):
        self.rom = rom
        self.reset()

    def run_cpu(self):
        i = 0
        while self.running:
            self.emulate_cycle()
            i += 1
            if i == 10:
                i = 0
                with objmode():
                    sleep(0.02)
                    
    def emulate_cycle(self):
        instruction = catbytes(self.memory[self.counter],
                               self.memory[self.counter+1])
        self.counter = chop(self.counter + 2, 12)
        self.dispatch(instruction)
        
    def dispatch(self, instruction, *args):
        @objmode
        getattr(self, self.instructions[instruction])(*args)
                    
    
            
        
        

SyntaxError: invalid syntax (<ipython-input-870-252fb69810fd>, line 110)

In [864]:
chip = ChipPure()

In [865]:
chip.rnd(1, 254)
print(chip.registers[1])

32


In [866]:
chip.registers[1]

32

In [867]:
chip.registers[0]

0

In [868]:
chip.dispatch(0xC000,2,0b1111111)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m- Resolution failure for literal arguments:
[1mFailed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'getattr':[0m [1m[1mCannot determine Numba type of <class 'builtin_function_or_method'>[0m
[1m
File "<ipython-input-863-3be968bdb369>", line 109:[0m
[1m    def dispatch(self, instruction, *args):
[1m        res = getattr(self, self.instructions[instruction])(*args)
[0m        [1m^[0m[0m
[0m[0m
[0m[1m- Resolution failure for non-literal arguments:
[1mNone[0m
[0m[0m
[0m[1mDuring: resolving callee type: BoundFunction((<class 'numba.core.types.misc.ClassInstanceType'>, 'dispatch') for instance.jitclass.ChipPure#7f1a30877220<running:bool,registers:array(int8, 1d, A),index:int16,counter:int16,stack:list(int16)<iv=None>,memory:array(int8, 1d, A),rom:unicode_type,flag:int8,cyclefreq:int8,cyclesleep:int8,instructions:DictType[int16,unicode_type]<iv=None>>)[

In [826]:
chip.stack

[0]

In [503]:
%%time
chip.loadrom('test')

CPU times: user 3.01 ms, sys: 0 ns, total: 3.01 ms
Wall time: 2.59 ms


In [513]:
chip.run_cpu()

2834
2836
2838
2840
2842
2844
2846
2848
2850
2852
2854
2856
2858
2860
2862
2864
2866
2868
2870
2872
2874
2876
2878
2880
2882
2884
2886
2888
2890
2892
2894
2896
2898
2900
2902
2904
2906
2908
2910
2912
2914
2916
2918
2920
2922
2924
2926
2928
2930
2932
2934
2936
2938
2940
2942
2944
2946
2948
2950
2952
2954
2956
2958
2960
2962
2964
2966
2968
2970
2972
2974
2976
2978
2980
2982
2984
2986
2988
2990
2992
2994
2996
2998
3000
3002
3004
3006
3008
3010
3012
3014
3016
3018
3020
3022
3024
3026
3028
3030
3032
3034
3036
3038
3040
3042
3044
3046
3048
3050
3052
3054
3056
3058
3060
3062
3064
3066
3068
3070
3072
3074
3076
3078
3080
3082
3084
3086
3088
3090
3092
3094
3096
3098
3100
3102
3104
3106
3108
3110
3112
3114
3116
3118
3120
3122
3124
3126
3128
3130
3132
3134
3136
3138
3140
3142
3144
3146
3148
3150
3152
3154
3156
3158
3160
3162
3164
3166
3168
3170
3172
3174
3176
3178
3180
3182
3184
3186
3188
3190
3192
3194
3196
3198
3200
3202
3204
3206
3208
3210
3212
3214
3216
3218
3220
3222
3224
3226
3228
3230
3232


2338
2340
2342
2344
2346
2348
2350
2352
2354
2356
2358
2360
2362
2364
2366
2368
2370
2372
2374
2376
2378
2380
2382
2384
2386
2388
2390
2392
2394
2396
2398
2400
2402
2404
2406
2408
2410
2412
2414
2416
2418
2420
2422
2424
2426
2428
2430
2432
2434
2436
2438
2440
2442
2444
2446
2448
2450
2452
2454
2456
2458
2460
2462
2464
2466
2468
2470
2472
2474
2476
2478
2480
2482
2484
2486
2488
2490
2492
2494
2496
2498
2500
2502
2504
2506
2508
2510
2512
2514
2516
2518
2520
2522
2524
2526
2528
2530
2532
2534
2536
2538
2540
2542
2544
2546
2548
2550
2552
2554
2556
2558
2560
2562
2564
2566
2568
2570
2572
2574
2576
2578
2580
2582
2584
2586
2588
2590
2592
2594
2596
2598
2600
2602
2604
2606
2608
2610
2612
2614
2616
2618
2620
2622
2624
2626
2628
2630
2632
2634
2636
2638
2640
2642
2644
2646
2648
2650
2652
2654
2656
2658
2660
2662
2664
2666
2668
2670
2672
2674
2676
2678
2680
2682
2684
2686
2688
2690
2692
2694
2696
2698
2700
2702
2704
2706
2708
2710
2712
2714
2716
2718
2720
2722
2724
2726
2728
2730
2732
2734
2736


1842
1844
1846
1848
1850
1852
1854
1856
1858
1860
1862
1864
1866
1868
1870
1872
1874
1876
1878
1880
1882
1884
1886
1888
1890
1892
1894
1896
1898
1900
1902
1904
1906
1908
1910
1912
1914
1916
1918
1920
1922
1924
1926
1928
1930
1932
1934
1936
1938
1940
1942
1944
1946
1948
1950
1952
1954
1956
1958
1960
1962
1964
1966
1968
1970
1972
1974
1976
1978
1980
1982
1984
1986
1988
1990
1992
1994
1996
1998
2000
2002
2004
2006
2008
2010
2012
2014
2016
2018
2020
2022
2024
2026
2028
2030
2032
2034
2036
2038
2040
2042
2044
2046
2048
2050
2052
2054
2056
2058
2060
2062
2064
2066
2068
2070
2072
2074
2076
2078
2080
2082
2084
2086
2088
2090
2092
2094
2096
2098
2100
2102
2104
2106
2108
2110
2112
2114
2116
2118
2120
2122
2124
2126
2128
2130
2132
2134
2136
2138
2140
2142
2144
2146
2148
2150
2152
2154
2156
2158
2160
2162
2164
2166
2168
2170
2172
2174
2176
2178
2180
2182
2184
2186
2188
2190
2192
2194
2196
2198
2200
2202
2204
2206
2208
2210
2212
2214
2216
2218
2220
2222
2224
2226
2228
2230
2232
2234
2236
2238
2240


1346
1348
1350
1352
1354
1356
1358
1360
1362
1364
1366
1368
1370
1372
1374
1376
1378
1380
1382
1384
1386
1388
1390
1392
1394
1396
1398
1400
1402
1404
1406
1408
1410
1412
1414
1416
1418
1420
1422
1424
1426
1428
1430
1432
1434
1436
1438
1440
1442
1444
1446
1448
1450
1452
1454
1456
1458
1460
1462
1464
1466
1468
1470
1472
1474
1476
1478
1480
1482
1484
1486
1488
1490
1492
1494
1496
1498
1500
1502
1504
1506
1508
1510
1512
1514
1516
1518
1520
1522
1524
1526
1528
1530
1532
1534
1536
1538
1540
1542
1544
1546
1548
1550
1552
1554
1556
1558
1560
1562
1564
1566
1568
1570
1572
1574
1576
1578
1580
1582
1584
1586
1588
1590
1592
1594
1596
1598
1600
1602
1604
1606
1608
1610
1612
1614
1616
1618
1620
1622
1624
1626
1628
1630
1632
1634
1636
1638
1640
1642
1644
1646
1648
1650
1652
1654
1656
1658
1660
1662
1664
1666
1668
1670
1672
1674
1676
1678
1680
1682
1684
1686
1688
1690
1692
1694
1696
1698
1700
1702
1704
1706
1708
1710
1712
1714
1716
1718
1720
1722
1724
1726
1728
1730
1732
1734
1736
1738
1740
1742
1744


KeyboardInterrupt: 

In [517]:
chr(0xF000)

'\uf000'